# [CptS 215 Data Analytics Systems and Algorithms](https://piazza.com/wsu/spring2017/cpts215/home)
[Washington State University](https://wsu.edu)

[Gina Sprint](http://eecs.wsu.edu/~gsprint/)
# L9-3 AVL Trees

Learner objectives for this lesson:
* Understand and implement balanced binary search trees (AVL trees)
* Discuss tree balance factors


## Acknowledgments
Content used in this lesson is based upon information in the following sources:
* [Miller and Ranum](http://interactivepython.org/runestone/static/pythonds/index.html)

## AVL (Balanced Binary Search) Trees
The efficiency of a BST decreases as the tree becomes less balanced. G.M. Adelson-Velskii and E.M. Landis designed a BST, called an AVL tree, that is balanced at all times. This improves the efficiency of the BST. 

To implement an AVL tree, we are going to keep track of a balance factor for each node in the tree. Upon inserting a node into the BST, we will consult tree's balance factors to properly insert the node while maintaining balance.

### Balance Factor
Each node in the tree is going to have a balance factor ($BF$) associated with it. The balance factor represents the difference between the height of the node's left subtree and the node's right subtree:

$$BF_{node} = height(leftsubtree) - height(rightsubtree)$$

A tree is balanced if its root node's balance factor is 0, left-heavy if it is positive, and right-heavy if it is negative. When $|BF| > 1$, we will re-balance the tree.

###  AVL Tree Insertion
When a new node is inserted into a BST, it is inserted as a leaf node. In an AVL tree, after the leaf node is inserted, we need to update the balance factors of the node's ancestors, until a balance factor is updated to 0 (the subtree is now balanced, no additional height was added by inserting the new node) or the root node has been reached. 

### AVL Tree Rebalancing
If a subtree is out balance enough to require rebalancing ($|BF| > 1$), we will bring the subtree back into balance by performing one ore more *rotations* on the tree. A rotation is either a left rotation (for a right-heavy subtree, $BF < -1$) or a right rotation (for a left-heavy subtree, $BF > 1$). 

Consider an example where the root (key: A) of a subtree has $BF = -2$. A's right child (key: B) has a right child (key: C) as well.

Algorithm for performing a left rotation:
1. Promote the right child (key: B) to the root of the subtree
1. Move the old root node (key: A) to the left child of the new root node (key: B)
1. If the new root node (key: B) already had a left child, then make it the right child of the new left child (key: A)

Consider an example where the root (key: A) of a subtree has $BF = 2$. A's left child (key: B) has a left child (key: C) as well.

Algorithm for performing a right rotation:
1. Promote the left child (key: B) to the root of the subtree
1. Move the old root node (key: A) to the right child of the new root node (key: B)
1. If the new root node (key: B) already had a right child, then make it the left child of the new right child (key: A)

In [63]:
class BSTNode:
    '''
    
    '''
    def __init__(self, key, val, left=None, right=None, parent=None):
        '''
        
        '''
        self.key = key
        self.value = val
        self.left_child = left
        self.right_child = right
        self.parent = parent
        self.balance_factor = 0
        
    def __iter__(self):
        '''
        Yield freezes the state of the function so that the next time the function 
        is called it continues executing from the exact point it left off earlier.
        '''
        if self:
            if self.has_left_child():
                 for elem in self.left_child:
                    yield elem
            yield self.key
            if self.has_right_child():
                 for elem in self.right_child:
                    yield elem

    def has_left_child(self):
        '''
        
        '''
        return self.left_child

    def has_right_child(self):
        '''
        
        '''
        return self.right_child

    def is_left_child(self):
        '''
        
        '''
        return self.parent and self.parent.left_child == self

    def is_right_child(self):
        '''
        
        '''
        return self.parent and self.parent.right_child == self

    def is_root(self):
        '''
        
        '''
        return not self.parent

    def is_leaf(self):
        '''
        
        '''
        return not (self.right_child or self.left_child)

    def has_any_children(self):
        '''
        
        '''
        return self.right_child or self.left_child

    def has_both_children(self):
        '''
        
        '''
        return self.right_child and self.left_child

    def replace_node_data(self, key, value, lc, rc):
        '''
        
        '''
        self.key = key
        self.value = value
        self.left_child = lc
        self.right_child = rc
        if self.has_left_child():
            self.left_child.parent = self
        if self.has_right_child():
            self.right_child.parent = self
            
class BinarySearchTree:
    '''
    
    '''
    def __init__(self):
        '''
        
        '''
        self.root = None
        self.size = 0

    def length(self):
        '''
        
        '''
        return self.size

    def __len__(self):
        '''
        Return the number of key-value pairs stored in the map.
        '''
        return self.size

    def __iter__(self):
        '''
        
        '''
        return self.root.__iter__()
    
    def __setitem__(self, k, v):
        '''
        
        '''
        self.put(k,v)
        
    def __getitem__(self, key):
        '''
        
        '''
        return self.get(key)
    
    def __delitem__(self,key):
        '''
        Delete the key-value pair from the map using a statement of the form del map[key].
        '''
        self.delete(key)
    
    def __contains__(self, key):
        '''
        Return True for a statement of the form key in map, if the given key is in the map.
        '''
        if self._get(key, self.root):
            return True
        else:
            return False
    
    def put(self, key, val):
        '''
        Add a new key-value pair to the map. 
        If the key is already in the map then replace the old value with the new value.
        
        If there is not a root then put will create a new TreeNode and install it as the root of the tree. 
        If a root node is already in place then put calls the private, recursive, helper function _put to search the tree
        '''
        if self.root:
            self._put(key, val, self.root)
        else:
            self.root = BSTNode(key, val)
        self.size = self.size + 1

    def _put(self, key, val, current_node):
        '''
        Starting at the root of the tree, search the binary tree comparing the new key 
        to the key in the current node. If the new key is less than the current node, 
        search the left subtree. If the new key is greater than the current node, 
        search the right subtree.

        When there is no left (or right) child to search, we have found the position 
        in the tree where the new node should be installed.

        To add a node to the tree, create a new TreeNode object and insert the object 
        at the point discovered in the previous step.
        '''
        if key == current_node.key: # handle duplicates by updating value
            current_node.value = val
        elif key < current_node.key:
            if current_node.has_left_child():
                self._put(key, val, current_node.left_child)
            else:
                current_node.left_child = BSTNode(key, val, parent=current_node)
        else:
            if current_node.has_right_child():
                self._put(key, val, current_node.right_child)
            else:
                current_node.right_child = BSTNode(key, val, parent=current_node)
                    
    def get(self, key):
        '''
        Given a key, return the value stored in the map or None otherwise.
        '''
        if self.root:
            res = self._get(key, self.root)
            if res:
                   return res.value
            else:
                   return None
        else:
            return None

    def _get(self, key, current_node):
        '''
        searches the tree recursively until it gets to a non-matching leaf node or finds a matching key. 
        When a matching key is found, the value stored in the payload of the node is returned.
        '''
        if not current_node:
            return None
        elif current_node.key == key:
            return current_node
        elif key < current_node.key:
            return self._get(key, current_node.left_child)
        else:
            return self._get(key, current_node.right_child)

    def delete(self, key):
        '''
        Find the node to delete by searching the tree using the 
        _get method to find the TreeNode that needs to be removed. 
        
        If the tree only has a single node, that means we are removing the root of the tree, 
        but we still must check to make sure the key of the root matches the key that is to be deleted.
        
        '''
        if self.size > 1:
            node_to_remove = self._get(key, self.root)
            if node_to_remove:
                self.remove(node_to_remove)
                self.size = self.size-1
            else:
                raise KeyError('Error, key not in tree')
        elif self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        else:
            raise KeyError('Error, key not in tree')

    def __delitem__(self,key):
        '''
        
        '''
        self.delete(key)

    def splice_out(self):
        '''
        Go directly to the node we want to splice out and makes the right changes.
        Handles case 1 and 2 of delete()
        No need to search for node to delete (unlike delete())
        '''
        if self.is_leaf(): # delete() case 1 (no children)
            if self.is_left_child():
                self.parent.left_child = None
            else:
                self.parent.right_child = None
        elif self.has_any_children(): # delete case 2 (exactly one child)
            if self.has_left_child():
                if self.is_left_child():
                    self.parent.left_child = self.left_child
                else:
                    self.parent.right_child = self.left_child
                self.left_child.parent = self.parent
            else:
                if self.is_left_child():
                    self.parent.left_child = self.right_child
                else:
                    self.parent.right_child = self.right_child
                self.right_child.parent = self.parent

    def find_successor(self):
        '''
        3 cases when looking for the successor:
        
        1. If the node has a right child, then the successor is the smallest key in the right subtree.
        2. If the node has no right child and is the left child of its parent, then the parent is the successor.
        3. If the node is the right child of its parent, and itself has no right child, 
        then the successor to this node is the successor of its parent, excluding this node.
        '''
        succ = None
        if self.has_right_child():
            succ = self.right_child.find_min()
        else:
            if self.parent:
                if self.is_left_child():
                    succ = self.parent
                else:
                    self.parent.right_child = None
                    succ = self.parent.find_successor()
                    self.parent.right_child = self
        return succ

    def find_min(self):
        '''
        Find the minimum key in a subtree.
        Left-most child in the subtree.
        '''
        current = self
        while current.has_left_child():
            current = current.left_child
        return current

    def remove(self, current_node):
        '''
        3 cases to consider:
        1. The node to be deleted has no children
        --->Delete the node and remove the reference to this node in the parent.
        2. The node to be deleted has only one child
        -->Promote the child to take the place of its parent.
        -->If the current node has no parent, it must be the root. Replace the key, value, left_child, 
        and right_child data by calling the replace_node_data method on the root.
        3. The node to be deleted has two 
        -->Search the tree for a node (successor) that can be used to replace the one scheduled for deletion
        -->Remove the successor and put it in the tree in place of the node to be deleted.
        '''
        if current_node.is_leaf(): #leaf (case 1)
            if current_node == current_node.parent.left_child:
                current_node.parent.left_child = None
            else:
                current_node.parent.right_child = None
        elif current_node.has_both_children(): #interior (case 3)
            succ = current_node.find_successor()
            succ.splice_out()
            current_node.key = succ.key
            current_node.value = succ.value
        else: # this node has one child (case 2)
            if current_node.has_left_child():
                if current_node.is_left_child():
                    current_node.left_child.parent = current_node.parent
                    current_node.parent.left_child = current_node.left_child
                elif currentNode.is_right_child():
                    current_node.left_child.parent = current_node.parent
                    current_node.parent.right_child = current_node.left_child
                else:
                    current_node.replace_node_data(current_node.left_child.key,
                                    current_node.left_child.value,
                                    current_node.left_child.left_child,
                                    current_node.left_child.right_child)
            else:
                if current_node.is_left_child():
                    current_node.right_child.parent = current_node.parent
                    current_node.parent.left_child = current_node.right_child
                elif current_node.is_right_child():
                    current_node.right_child.parent = current_node.parent
                    current_node.parent.right_child = current_node.right_child
                else:
                    current_node.replace_node_data(current_node.right_child.key,
                                    current_node.right_child.value,
                                    current_node.right_child.left_child,
                                    current_node.right_child.right_child)
    def in_order_traversal(self):
        '''
        
        '''
        if self.size > 0:
            self.in_order_helper(self.root)
            print()
        else:
            print("Empty tree")
            
    def in_order_helper(self, current_node):
        '''
        
        '''
        if current_node is not None:
            self.in_order_helper(current_node.left_child)
            print(str(current_node.key) + ":" + str(current_node.value), end=" ")
            self.in_order_helper(current_node.right_child)
            
class AVLTree(BinarySearchTree):
    '''
    
    '''
    def __init__(self):
        '''
        
        '''
        super(AVLTree, self).__init__()
        
    def _put(self, key, val, current_node):
        '''
        
        '''
        if key == current_node.key: # handle duplicates by updating value
            current_node.value = val
        elif key < current_node.key:
            if current_node.has_left_child():
                self._put(key, val, current_node.left_child)
            else:
                current_node.left_child = BSTNode(key, val, parent=current_node)
                self.update_balance(current_node.left_child)
        else:
            if current_node.has_right_child():
                self._put(key, val, current_node.right_child)
            else:
                current_node.right_child = BSTNode(key, val, parent=current_node)
                self.update_balance(current_node.right_child)

    def update_balance(self, node):
        '''
        
        '''
        if node.balance_factor > 1 or node.balance_factor < -1:
            self.rebalance(node)
            return
        if node.parent != None:
            if node.is_left_child():
                node.parent.balance_factor += 1
            elif node.is_right_child():
                node.parent.balance_factor -= 1

            if node.parent.balance_factor != 0:
                self.update_balance(node.parent)
                
    def rebalance(self, node):
        '''
        
        '''
        if node.balance_factor < 0:
            if node.right_child.balance_factor > 0:
                self.rotate_right(node.right_child)
                self.rotate_left(node)
            else:
                self.rotate_left(node)
        elif node.balance_factor > 0:
            if node.left_child.balance_factor < 0:
                self.rotate_left(node.left_child)
                self.rotate_right(node)
            else:
                self.rotate_right(node)
                
    def rotate_left(self, rot_root):
        '''
        
        '''
        new_root = rot_root.right_child
        rot_root.right_child = new_root.left_child
        if new_root.left_child != None:
            newRoot.left_child.parent = rot_root
        new_root.parent = rot_root.parent
        if rot_root.is_root():
            self.root = new_root
        else:
            if rot_root.is_left_child():
                rot_root.parent.left_child = new_root
            else:
                rot_root.parent.right_child = new_root
        new_root.left_child = rot_root
        rot_root.parent = new_root
        rot_root.balance_factor = rot_root.balance_factor + 1 - min(new_root.balance_factor, 0)
        new_root.balance_factor = new_root.balance_factor + 1 + max(rot_root.balance_factor, 0)
        
    def rotate_right(self, rot_root):
        '''
        
        '''
        new_root = rot_root.left_child
        rot_root.left_child = new_root.right_child
        if new_root.right_child != None:
            new_root.right_child.parent = rot_root
        new_root.parent = rot_root.parent
        if rot_root.is_root():
            self.root = new_root
        else:
            if rot_root.is_right_child():
                rot_root.parent.right_child = new_root
            else:
                rot_root.parent.left_child = new_root
        new_root.right_child = rot_root
        rot_root.parent = new_root
        rot_root.balance_factor = rot_root.balance_factor - 1 - max(new_root.balance_factor, 0)
        new_root.balance_factor = new_root.balance_factor - 1 + min(rot_root.balance_factor, 0)
            
    def in_order_helper(self, current_node):
        '''
        
        '''
        if current_node is not None:
            self.in_order_helper(current_node.left_child)
            print(str(current_node.key) + ":" + str(current_node.value) + "(%d)" %(current_node.balance_factor), end=" ")
            self.in_order_helper(current_node.right_child)
            
    def level_order_traversal(self):
        '''
        
        '''
        if self.size > 0:
            queue = [str(self.root.key)+":"+str(self.root.value) +"(%d)" %(self.root.balance_factor), "\n"]
            self.level_order_helper(self.root, queue)
            for data in queue:
                print(data, end="")
            print()
        else:
            print("Empty tree")
            
    def level_order_helper(self, node, queue):
        '''
        
        '''
        if node is not None:
            if node.left_child is not None:
                temp = node.left_child
                queue.append(str(temp.key)+":"+str(temp.value) +"(%d)" %(temp.balance_factor))
            if node.right_child is not None:
                temp = node.right_child
                queue.append(str(temp.key)+":"+str(temp.value) +"(%d)" %(temp.balance_factor))
            queue.append("\n")
            self.level_order_helper(node.left_child, queue)
            self.level_order_helper(node.right_child, queue)
            
mytree = AVLTree()
mytree[7]="town"
mytree[6]="at"
mytree[5]="yellow"
mytree[4]="blue"
mytree[3]="red"
'''
mytree[3]="red"
mytree[4]="blue"
mytree[5]="yellow"
mytree[6]="at"
mytree[7]="town"'''

mytree.level_order_traversal()
#del mytree[6]
#mytree.level_order_traversal()

6:at(1)
4:blue(0)7:town(0)
3:red(0)5:yellow(0)






## Practice Problems
Note: the following problems are adapted from Koffman and Wolfgang.

### 1
Show how the final AVL tree for inserting the words in the sentence, "The quick brown fox" changes as you insert "apple", "cat", and "hat" in that order.

### 2
Build an AVL tree that results from inserting the integers 30, 40, 15, 25, 90, 80, 70, 85, 15, 72 in the given order.

### 3
Build the AVL tree that results from inserting the words in the sentence, "Now is the time for all good men to come to the aid of the party."